<a href="https://colab.research.google.com/github/markelgarcia282/03MIAR----Algoritmos-de-Optimizacion/blob/main/AG3/Markel_Garcia_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nombre: Markel Garcia Ibarluzea

Github: https://github.com/markelgarcia282/03MIAR----Algoritmos-de-Optimizacion

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import tsplib95
import urllib.request
import random
from copy import deepcopy

In [ ]:
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
#!gzip -d swiss42.tsp.gz

('swiss42.tsp.gz', <http.client.HTTPMessage at 0x7f0b37d99110>)

In [ ]:
problema = tsplib95.load(file)

#Nodos
Nodos = list(problema.get_nodes())

#Aristas
Aristas = list(problema.get_edges())

In [ ]:
print(problema)

NAME: swiss42
COMMENT: 42 Staedte Schweiz (Fricker)
TYPE: TSP
DIMENSION: 42
EDGE_WEIGHT_TYPE: EXPLICIT
EDGE_WEIGHT_FORMAT: FULL_MATRIX
EDGE_WEIGHT_SECTION:
0 15 30 23 32 55 33 37 92 114 92 110 96 90 74 76 82 67 72 78 82 159 122 131 206 112 57 28 43 70 65 66 37 103 84 125 129 72 126 141 183 124
15 0 34 23 27 40 19 32 93 117 88 100 87 75 63 67 71 69 62 63 96 164 132 131 212 106 44 33 51 77 75 72 52 118 99 132 132 67 139 148 186 122
30 34 0 11 18 57 36 65 62 84 64 89 76 93 95 100 104 98 57 88 99 130 100 101 179 86 51 4 18 43 45 95 45 115 93 152 159 100 112 114 153 94
23 23 11 0 11 48 26 54 70 94 69 89 75 84 84 89 92 89 54 78 99 141 111 109 190 89 44 11 29 54 56 89 47 118 96 147 151 90 122 126 163 101
32 27 18 11 0 40 20 58 67 92 61 78 65 76 83 89 91 95 43 72 110 141 116 105 190 81 34 19 35 57 63 97 58 129 107 156 158 92 129 127 161 95
55 40 57 48 40 0 23 55 96 123 78 75 62 36 56 66 63 95 37 34 137 174 156 129 224 90 15 59 75 96 103 105 91 158 139 164 156 78 169 163 191 115
33 19 36 26 20 

In [ ]:
# Función que genera una solución aleatoria al problema del viajante que empieza y termina en 0.
def crear_solucion(problema):
    
    nodos = list(problema.get_nodes())
    
    solucion = nodos[1:]
    random.shuffle(solucion)
    
    return [0] + solucion + [0]

# Problema que calcula el coste de una solución, donde el coste es la suma de las distancias que el viajero recorre.
def calcular_coste(solucion, problema):
    
    return sum([problema.get_weight(a, b) for a, b in zip(solucion[:-1], solucion[1:])])

## Busqueda Aleatoria

In [ ]:
def busqueda_aleatoria(problema, num_pruebas):
    
    # Inicializamos la mejor solución a un valor alto.
    mejor_coste = math.inf
    
    for _ in range(num_pruebas):
        
        # Generamos solución aleatoria y calculamos su coste
        solucion_prueba = crear_solucion(problema)
        coste_prueba = calcular_coste(solucion_prueba, problema)
        
        # Si la solución generada tiene menor coste que la mejor encontrada hasta el momento, actualizamos la mejor solución.
        if coste_prueba < mejor_coste:
            
            mejor_solucion = solucion_prueba
            mejor_coste = coste_prueba
            
    return mejor_solucion, mejor_coste
        

In [ ]:
mejor_solucion, mejor_coste = busqueda_aleatoria(problema, 5000)
print(f'Mejor solución: {mejor_solucion}\nDistancia total en la mejor solución: {mejor_coste}')

Mejor solución: [0, 31, 20, 16, 7, 14, 33, 1, 18, 8, 22, 24, 21, 29, 5, 2, 23, 28, 37, 25, 39, 10, 40, 41, 12, 3, 9, 11, 27, 4, 19, 6, 26, 36, 35, 34, 38, 32, 17, 30, 15, 13, 0]
Distancia total en la mejor solución: 3698


## Busqueda Local

In [ ]:
def generar_vecina(solucion, problema):
    
    mejor_coste = math.inf
    
    # Para cada nodo, recorremos el resto de nodos y probamos intercambios entre pares de nodos.
    for i in range(1, len(solucion)-1):
        for j in range(i+1, len(solucion)):
            
            # Intercambiamos nodos i y j.
            vecina = deepcopy(solucion)
            vecina[i], vecina[j] = solucion[j], solucion[i]
            
            # Calculamos coste de nueva solución.
            nuevo_coste = calcular_coste(vecina, problema)
            
            # Si la solución generada tiene menor coste que la mejor encontrada hasta el momento, actualizamos la mejor solución.
            if nuevo_coste < mejor_coste:
            
                mejor_solucion = vecina
                mejor_coste = nuevo_coste
    
    return mejor_solucion, mejor_coste

In [ ]:
print(f'Distancia total solución inicial: {mejor_coste}')

mejor_solucion, mejor_coste = generar_vecina(mejor_solucion, problema)
print(f'Distancia total mejor solución local: {mejor_coste}')

Distancia total solución inicial: 3698
Distancia total mejor solución local: 3336


In [ ]:
def busqueda_local(problema, operador):
    
    # Generamos una solución inicial de referencia y calculamos su coste.
    solucion_referencia = crear_solucion(problema)
    mejor_coste = calcular_coste(solucion_referencia, problema)
    
    # Contador de iteraciones.
    iters = 0
    
    while True:
        
        iters += 1
        
        # Obtenemos la major vecina de la solución referencia y su coste
        mejor_vecina, coste_vecina = operador(solucion_referencia, problema)
        
        # Comprobamos si mejora el mejor coste actual. Si no, detenemos el algoritmo ya que hemos llegado a un mínimo local.
        if coste_vecina < mejor_coste:
            
            mejor_coste = coste_vecina
            mejor_solucion = mejor_vecina
            solucion_referencia = mejor_vecina
            
        else:
            return mejor_solucion, mejor_coste, iters

In [ ]:
mejor_solucion, mejor_coste, iters = busqueda_local(problema, generar_vecina)

print(f'En la iteracion: {iters}, la solución mejor encontrada es: {mejor_solucion}\ncon distancia total: {mejor_coste}')

En la iteracion: 30, la solución mejor encontrada es: [0, 6, 26, 5, 13, 19, 7, 37, 15, 16, 14, 11, 25, 41, 23, 9, 8, 29, 30, 28, 32, 34, 33, 20, 35, 36, 31, 17, 18, 12, 10, 2, 27, 3, 4, 1, 0, 38, 22, 39, 21, 40, 24]
con distancia total: 1594


Consideramos ahora como cambiaría el rendimiento del algoritmo haciendo uso de diferentes operadores para generar nuevas soluciones.

In [ ]:
def generar_insercion(solucion, problema):
    
    mejor_coste = math.inf
    
    # Para cada nodo, recorremos el resto de nodos y probamos intercambios entre pares de nodos.
    for i in range(1, len(solucion)-1):
        for j in range(i+1, len(solucion)):
            
            # Mover el nodo i a la posición j.
            nueva_solucion = deepcopy(solucion)
            nueva_solucion.insert(j, nueva_solucion.pop(i))
            
            # Calculamos coste de nueva solución.
            nuevo_coste = calcular_coste(nueva_solucion, problema)
            
            # Si la solución generada tiene menor coste que la mejor encontrada hasta el momento, actualizamos la mejor solución.
            if nuevo_coste < mejor_coste:
            
                mejor_solucion = nueva_solucion
                mejor_coste = nuevo_coste
    
    return mejor_solucion, mejor_coste

In [ ]:
mejor_solucion, mejor_coste, iters = busqueda_local(problema, generar_insercion)

print(f'En la iteracion: {iters}, la solución mejor encontrada es: {mejor_solucion}\ncon distancia total: {mejor_coste}')

En la iteracion: 35, la solución mejor encontrada es: [0, 20, 33, 34, 38, 22, 24, 40, 23, 41, 25, 11, 12, 18, 27, 32, 35, 36, 37, 15, 16, 14, 19, 13, 5, 26, 6, 3, 2, 28, 30, 29, 39, 21, 9, 8, 10, 4, 1, 0, 7, 17, 31]
con distancia total: 1604


In [ ]:
def generar_inversion(solucion, problema, rango=4):
    
    mejor_coste = math.inf
    
    # Para cada nodo, recorremos el resto de nodos y probamos intercambios entre pares de nodos.
    for i in range(1, len(solucion)-rango+1):
            
        # Invertir el orden de la sublista que empieza en el elemento i y tiene una longitud igual a rango.
        nueva_solucion = deepcopy(solucion)
        nueva_solucion[i:i+rango] = reversed(nueva_solucion[i:i+rango])
                
        # Calculamos coste de nueva solución.
        nuevo_coste = calcular_coste(nueva_solucion, problema)
        
        # Si la solución generada tiene menor coste que la mejor encontrada hasta el momento, actualizamos la mejor solución.
        if nuevo_coste < mejor_coste:
        
            mejor_solucion = nueva_solucion
            mejor_coste = nuevo_coste
    
    return mejor_solucion, mejor_coste
    

In [ ]:
mejor_solucion, mejor_coste, iters = busqueda_local(problema, generar_inversion)

print(f'En la iteracion: {iters}, la solución mejor encontrada es: {mejor_solucion}\ncon distancia total: {mejor_coste}')

En la iteracion: 21, la solución mejor encontrada es: [0, 20, 17, 26, 25, 8, 38, 9, 39, 3, 4, 2, 31, 32, 40, 11, 12, 7, 16, 15, 19, 13, 1, 5, 6, 27, 33, 30, 28, 18, 10, 41, 24, 23, 21, 34, 29, 22, 0, 37, 35, 36, 14]
con distancia total: 3162


In [ ]:
def generar_aleatoria(solucion, problema, rango=4):
    
    mejor_coste = math.inf
    
    # Para cada nodo, recorremos el resto de nodos y probamos intercambios entre pares de nodos.
    for i in range(1, len(solucion)-rango+1):
            
        # Cambiar aleatoriamente el orden de la sublista que empieza en el elemento i y tiene una longitud igual a rango.
        nueva_solucion = deepcopy(solucion)
        sub_lista = nueva_solucion[i:i+rango]
        random.shuffle(sub_lista)
        nueva_solucion[i:i+rango] = sub_lista
                
        # Calculamos coste de nueva solución.
        nuevo_coste = calcular_coste(nueva_solucion, problema)
        
        # Si la solución generada tiene menor coste que la mejor encontrada hasta el momento, actualizamos la mejor solución.
        if nuevo_coste < mejor_coste:
        
            mejor_solucion = nueva_solucion
            mejor_coste = nuevo_coste
    
    return mejor_solucion, mejor_coste
    

In [ ]:
mejor_solucion, mejor_coste, iters = busqueda_local(problema, generar_aleatoria)

print(f'En la iteracion: {iters}, la solución mejor encontrada es: {mejor_solucion}\ncon distancia total: {mejor_coste}')

En la iteracion: 20, la solución mejor encontrada es: [0, 31, 36, 8, 12, 6, 37, 14, 13, 11, 9, 3, 1, 7, 15, 4, 27, 22, 30, 32, 33, 20, 35, 19, 40, 24, 39, 23, 25, 41, 26, 16, 29, 38, 34, 28, 2, 21, 10, 5, 18, 0, 17]
con distancia total: 3232


Vemos que el operador de inserción es el que mejor resultado obtiene, mejorando el rendimiento del operador de vecindad que implementamos inicialmente. Los operadores de inversión y orden aleatorio tienen bastante peor rendimiento, aunque la comparación no es del todo justa ya que probamos menos soluciones con estas. En los otros operadores realizamos un doble bucle probando una solución para cada par de índices mientras que en estos dos operadores, solo iteramos sobre los índices de la lista una vez. Es posible que el rendimiento mejore si permitimos que la longitud de la sublista que invertimos/barajamos cambie también.

También es posible combinar varios de estos operadores, con el objetivo de escapar mínimos locales. La idea, como se explica en "Búsqueda por Entornos Variables para Planificación Logística", es que el mínimo local asociado a un entorno local no es necesariamente un mínimo local en otro entorno, por lo que cambiar de entorno durante el proceso de búsqueda puede ayudarnos a encontrar en mínimo global. En este caso, aplicaremos como prueba uno de los algoritmos que implementan esta idea en el artículo: VNS Descendente.

In [ ]:
operadores = [generar_aleatoria, generar_inversion, generar_vecina, generar_insercion]

In [ ]:
def VNS_descendente(problema, operadores):
    
    # Generamos solución inicial y calculamos su coste.
    mejor_solucion = crear_solucion(problema)
    mejor_coste = calcular_coste(mejor_solucion, problema)
    
    k = 0
    
    while k < len(operadores):
        
        print(f'k: {k}, mejor solucion: {mejor_coste}')
        
        # Buscamos solución óptima para entorno k.
        nueva_solucion, nuevo_coste, _ = busqueda_local(problema, operadores[k])
        
        # Comprobamos si mejora la solución óptima hasta ahora. Si lo hace, volvemos a iterar sobre los entornos. Si no, pasamos al siguiente entorno.
        if nuevo_coste < mejor_coste:
            
            mejor_coste = nuevo_coste
            mejor_solucion = nueva_solucion
            k = 0
            
        else:
            
            k += 1
            
    return mejor_solucion, mejor_coste

In [ ]:
mejor_solucion, mejor_coste = VNS_descendente(problema, operadores)

print(f'La solución mejor encontrada es: {mejor_solucion}\ncon distancia total: {mejor_coste}')

k: 0, mejor solucion: 4869
k: 0, mejor solucion: 3196
k: 0, mejor solucion: 3052
k: 1, mejor solucion: 3052
k: 2, mejor solucion: 3052
k: 0, mejor solucion: 1864
k: 1, mejor solucion: 1864
k: 2, mejor solucion: 1864
k: 0, mejor solucion: 1546
k: 1, mejor solucion: 1546
k: 2, mejor solucion: 1546
k: 3, mejor solucion: 1546
k: 0, mejor solucion: 1506
k: 1, mejor solucion: 1506
k: 2, mejor solucion: 1506
k: 3, mejor solucion: 1506
La solución mejor encontrada es: [0, 20, 33, 34, 38, 22, 30, 28, 2, 5, 13, 19, 14, 16, 15, 37, 17, 36, 35, 31, 32, 3, 4, 6, 26, 18, 12, 11, 25, 10, 41, 23, 40, 24, 21, 39, 9, 8, 29, 27, 0, 1, 7]
con distancia total: 1506


## Simulated Annealing

In [ ]:
# Función que genera una solucion, intercambiando dos nodos aleatorios de una solución referencia.
def generar_vecina_aleatoria(solucion):
    
    # Escojemos dos nodos aleatorios para intercambiar.
    i, j = random.sample(range(1, len(solucion)-1), 2)
    
    # Intercambiamos los nodos y devolvemos la solucion.
    nueva_solucion = deepcopy(solucion)
    nueva_solucion[i], nueva_solucion[j] = solucion[j], solucion[i]
    
    return nueva_solucion

# Devuelve la probabilidad de aceptar una nueva solución dado el coste de la propuesta y el coste de referencia.
def criterio_aceptacion(T, coste_actual, coste_previo):
    
    delta = coste_actual - coste_previo
    prob_aceptar = 1.0 if delta <= 0 else math.exp(-(delta)/T)
    
    return prob_aceptar > random.random()

In [ ]:
def simulated_annealing(problema, temperatura, operador):
    
    mejor_coste = math.inf
    
    # Generamos una solución inicial de referencia y calculamos su coste.
    solucion_referencia = crear_solucion(problema)
    coste_referencia = calcular_coste(solucion_referencia, problema)
    
    while temperatura > 0.01:
                
        # Generamos vecino y calcular su coste
        nueva_solucion = operador(solucion_referencia)
        nuevo_coste = calcular_coste(nueva_solucion, problema)
        
        # Acualizamos la mejor solución si el coste es menor que el coste mínimo encontrado hasta ahora.
        if nuevo_coste < mejor_coste:
            
            mejor_coste = nuevo_coste
            mejor_solucion = nueva_solucion
            
        # Utilizamos el criterio de aceptación para determinar si aceptamos la nueva solución propuesta.append
        if criterio_aceptacion(temperatura, nuevo_coste, coste_referencia):
            
            coste_referencia = nuevo_coste
            solucion_referencia = nueva_solucion
            
        # Bajamos la temperatura.
        temperatura *= 0.99
        
    return mejor_solucion, mejor_coste

In [ ]:
mejor_solucion, mejor_coste = simulated_annealing(problema, 10000000, generar_vecina_aleatoria)

print(f'La mejor solución encontrada es: {mejor_solucion}\nCon una distancia total recorrida de: {mejor_coste}')

La mejor solución encontrada es: [0, 18, 10, 9, 39, 30, 29, 28, 31, 35, 36, 15, 13, 19, 14, 5, 6, 4, 26, 12, 11, 25, 41, 8, 2, 34, 33, 38, 22, 21, 24, 40, 23, 1, 7, 3, 27, 32, 20, 16, 37, 17, 0]
Con una distancia total recorrida de: 2213


El resultado podría optimizarse eligiendo un operador que no sea tan aleatorio, es decir, que mejore ligeramente la explotación a cambio de reducir la exploración del algoritmo, ya que como vimos con el algoritmo de búsqueda local, un operador no tan aleatorio como el operador de inserción podría mejorar el rendimiento. Probamos entonces con ese mismo operador:

In [ ]:
# Función que genera una solucion, intercambiando dos nodos aleatorios de una solución referencia.
def generar_insercion_aleatoria(solucion):
    
    # Escojemos un nodos aleatorio y lo movemos otra posición aleatoriamente escogida.
    i, j = random.sample(range(1, len(solucion)-1), 2)
    
    # Intercambiamos los nodos y devolvemos la solucion.
    nueva_solucion = deepcopy(solucion)
    nueva_solucion.insert(j, nueva_solucion.pop(i))
    
    return nueva_solucion

In [ ]:
mejor_solucion, mejor_coste = simulated_annealing(problema, 10000000, generar_insercion_aleatoria)

print(f'La mejor solución encontrada es: {mejor_solucion}\nCon una distancia total recorrida de: {mejor_coste}')

La mejor solución encontrada es: [0, 7, 16, 19, 13, 5, 10, 41, 8, 9, 23, 21, 40, 24, 39, 33, 34, 20, 31, 17, 37, 36, 35, 32, 27, 2, 4, 25, 11, 12, 18, 26, 15, 14, 6, 30, 38, 22, 29, 28, 3, 1, 0]
Con una distancia total recorrida de: 1883


Vemos que, efectivamente, mejoramos el resultado con una mejor elección del operador.